In [ ]:
# visualize_sentiment.py

import os
import sys
from pathlib import Path

import pandas as pd
import torch
import requests
import matplotlib.pyplot as plt
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# ─── resolve script_dir & add project root ────────────────────────────────────
try:
    script_dir = Path(__file__).parent
except NameError:
    script_dir = Path.cwd()

proj_root = script_dir.parent
if str(proj_root) not in sys.path:
    sys.path.insert(0, str(proj_root))

# ─── CONFIG ───────────────────────────────────────────────────────────────────
API_KEY    = os.getenv("FMP_API_KEY")
if not API_KEY:
    raise RuntimeError("Missing FMP_API_KEY environment variable. "
                       "Please set it to your FinancialModelingPrep API key.")

MODEL_PATH = script_dir / "finbert-finetuned-final"
ART_DIR    = script_dir / "articles"

# ─── HELPERS ──────────────────────────────────────────────────────────────────
def load_articles(ticker: str) -> pd.DataFrame:
    path = ART_DIR / f"{ticker}.csv"
    if not path.exists():
        raise FileNotFoundError(f"No article CSV found at {path}")
    df = pd.read_csv(path, parse_dates=["publishedDate"])
    df["date"] = df["publishedDate"].dt.date
    return df

def analyze_sentiment(df: pd.DataFrame, tokenizer, model) -> pd.DataFrame:
    recs = []
    for _, row in df.iterrows():
        text = row.get("content") or row.get("title", "")
        if not isinstance(text, str) or not text.strip():
            continue
        inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
        with torch.no_grad():
            logits = model(**inputs).logits
        probs = torch.softmax(logits, dim=1)[0]
        recs.append({
            "date": row["date"],
            "sentiment": probs[0].item() - probs[2].item()
        })
    return pd.DataFrame(recs)

def fetch_prices(ticker: str) -> pd.DataFrame:
    url = (
        f"https://financialmodelingprep.com/api/v3/"
        f"historical-price-full/{ticker}?apikey={API_KEY}"
    )
    resp = requests.get(url, timeout=10)
    if resp.status_code == 401:
        raise RuntimeError("Unauthorized fetching prices: check your FMP_API_KEY.")
    resp.raise_for_status()
    data = resp.json().get("historical", [])
    if not data:
        raise ValueError(f"No price history returned for {ticker}.")
    prices = pd.DataFrame(data)
    prices["date"] = pd.to_datetime(prices["date"])
    return prices[["date", "close"]]

# ─── MAIN ─────────────────────────────────────────────────────────────────────
def main(ticker: str):
    # 1. load model & tokenizer
    tokenizer = AutoTokenizer.from_pretrained(str(MODEL_PATH))
    model     = AutoModelForSequenceClassification.from_pretrained(str(MODEL_PATH))
    model.eval()

    # 2. score articles
    art_df  = load_articles(ticker)
    sent_df = analyze_sentiment(art_df, tokenizer, model)
    if sent_df.empty:
        print(f"No valid articles found for {ticker}.")
        return

    # 3. daily sentiment
    daily = (
        sent_df
        .groupby("date")["sentiment"]
        .mean()
        .reset_index()
        .assign(date=lambda df: pd.to_datetime(df["date"]))
    )

    # 4. fetch & merge prices
    price_df = fetch_prices(ticker)
    merged   = price_df.merge(daily, on="date", how="left")
    merged["sentiment"].fillna(0, inplace=True)
    merged["sentiment_ma"] = merged["sentiment"].rolling(7, min_periods=1).mean()

    # 5. plot
    fig, ax1 = plt.subplots(figsize=(14,7))
    ax1.plot(merged["date"], merged["close"], label="Close Price")
    ax1.set_xlabel("Date"); ax1.set_ylabel("Price")
    ax1.grid(True, linestyle="--", alpha=0.5)

    ax2 = ax1.twinx()
    ax2.plot(
        merged["date"],
        merged["sentiment_ma"],
        color="tab:red",
        label="7-Day Sentiment MA"
    )
    ax2.set_ylabel("Sentiment Score")
    ax2.axhline(0, color="gray", linestyle="--", linewidth=0.8)

    # combined legend & title
    h1, l1 = ax1.get_legend_handles_labels()
    h2, l2 = ax2.get_legend_handles_labels()
    ax1.legend(h1+h2, l1+l2, loc="upper left")
    plt.title(f"{ticker}: Price vs. Sentiment")
    plt.tight_layout()
    plt.show()

# ─── ENTRY POINT ──────────────────────────────────────────────────────────────
if __name__ == "__main__":
    ticker = input("Enter ticker (e.g. AAPL): ").strip().upper()
    try:
        main(ticker)
    except Exception as e:
        print(f"Error: {e}")


In [ ]:
# visualize_sentiment.py
import os
import sys
from pathlib import Path
from datetime import datetime

import pandas as pd
import torch
import requests
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# ─── resolve script_dir & add project root ────────────────────────────────────
try:
    script_dir = Path(__file__).parent
except NameError:
    script_dir = Path.cwd()

proj_root = script_dir.parent
if str(proj_root) not in sys.path:
    sys.path.insert(0, str(proj_root))

# ─── CONFIG ───────────────────────────────────────────────────────────────────
API_KEY = os.getenv("FMP_API_KEY")
if not API_KEY:
    raise RuntimeError("Missing FMP_API_KEY environment variable. "
                      "Please set it to your FinancialModelingPrep API key.")

MODEL_PATH = script_dir / "finbert-finetuned-final"
ART_DIR = script_dir / "articles"

# ─── HELPERS ──────────────────────────────────────────────────────────────────
def load_articles(ticker: str) -> pd.DataFrame:
    """Load news articles for given ticker"""
    path = ART_DIR / f"{ticker}.csv"
    if not path.exists():
        raise FileNotFoundError(f"No article CSV found at {path}")
    df = pd.read_csv(path, parse_dates=["publishedDate"])
    df["date"] = df["publishedDate"].dt.date
    return df

def analyze_sentiment(df: pd.DataFrame, tokenizer, model) -> pd.DataFrame:
    """Analyze sentiment of news articles"""
    recs = []
    for _, row in df.iterrows():
        text = row.get("content") or row.get("title", "")
        if not isinstance(text, str) or not text.strip():
            continue
        inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
        with torch.no_grad():
            logits = model(**inputs).logits
        probs = torch.softmax(logits, dim=1)[0]
        recs.append({
            "date": row["date"],
            "sentiment": probs[0].item() - probs[2].item()  # positive - negative
        })
    return pd.DataFrame(recs)

def fetch_prices(ticker: str) -> pd.DataFrame:
    """Fetch historical stock prices"""
    url = (
        f"https://financialmodelingprep.com/api/v3/"
        f"historical-price-full/{ticker}?apikey={API_KEY}"
    )
    resp = requests.get(url, timeout=10)
    if resp.status_code == 401:
        raise RuntimeError("Unauthorized fetching prices: check your FMP_API_KEY.")
    resp.raise_for_status()
    data = resp.json().get("historical", [])
    if not data:
        raise ValueError(f"No price history returned for {ticker}.")
    prices = pd.DataFrame(data)
    prices["date"] = pd.to_datetime(prices["date"])
    return prices[["date", "close"]]

def format_plot(ax1, ax2, ticker, sentiment_start, sentiment_end):
    """Format the plot with consistent styling"""
    # Set title and labels
    ax1.set_title(f"{ticker} Stock Price vs. News Sentiment", pad=20)
    
    # Format x-axis
    ax1.xaxis.set_major_locator(mdates.MonthLocator())
    ax1.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
    plt.setp(ax1.get_xticklabels(), rotation=45, ha='right')
    
    # Grid and colors
    ax1.grid(True, linestyle='--', alpha=0.3)
    ax1.set_ylabel('Stock Price', color='b')
    ax1.tick_params(axis='y', colors='b')
    
    ax2.set_ylabel('Sentiment Score (7-day MA)', color='r')
    ax2.tick_params(axis='y', colors='r')
    
    # Add reference lines
    ax2.axhline(0, color='gray', linestyle='--', linewidth=1, label='Neutral')
    
    # Add sentiment period markers if within plot range
    xlim = ax1.get_xlim()
    plot_start = mdates.num2date(xlim[0]).date()
    plot_end = mdates.num2date(xlim[1]).date()
    
    if isinstance(sentiment_start, datetime):
        sentiment_start = sentiment_start.date()
    if isinstance(sentiment_end, datetime):
        sentiment_end = sentiment_end.date()
    
    if sentiment_start >= plot_start and sentiment_start <= plot_end:
        ax2.axvline(pd.to_datetime(sentiment_start), 
                   color='g', linestyle=':', alpha=0.7, label='Sentiment Start')
    if sentiment_end >= plot_start and sentiment_end <= plot_end:
        ax2.axvline(pd.to_datetime(sentiment_end), 
                   color='r', linestyle=':', alpha=0.7, label='Sentiment End')

# ─── MAIN ─────────────────────────────────────────────────────────────────────
def main(ticker: str, start_date=None, end_date=None):
    """Main analysis and plotting function"""
    # 1. Load model and tokenizer
    print("Loading sentiment model...")
    tokenizer = AutoTokenizer.from_pretrained(str(MODEL_PATH))
    model = AutoModelForSequenceClassification.from_pretrained(str(MODEL_PATH))
    model.eval()

    # 2. Score articles
    print("Loading and analyzing articles...")
    try:
        art_df = load_articles(ticker)
    except FileNotFoundError as e:
        print(e)
        return

    sent_df = analyze_sentiment(art_df, tokenizer, model)
    if sent_df.empty:
        print(f"No valid articles found for {ticker}.")
        return

    sentiment_start = sent_df['date'].min()
    sentiment_end = sent_df['date'].max()

    # 3. Calculate daily sentiment
    print("Calculating daily sentiment...")
    daily = (
        sent_df
        .groupby("date")["sentiment"]
        .mean()
        .reset_index()
        .assign(date=lambda df: pd.to_datetime(df["date"]))
    )

    # 4. Fetch and merge prices
    print("Fetching price data...")
    try:
        price_df = fetch_prices(ticker)
    except Exception as e:
        print(f"Error fetching prices: {e}")
        return

    merged = price_df.merge(daily, on="date", how="left")
    merged["sentiment"].fillna(0, inplace=True)  # Neutral for days without articles
    merged["sentiment_ma"] = merged["sentiment"].rolling(7, min_periods=1).mean()

    # Filter by date range if specified
    if start_date:
        start_date = pd.to_datetime(start_date)
        merged = merged[merged['date'] >= start_date]
    if end_date:
        end_date = pd.to_datetime(end_date)
        merged = merged[merged['date'] <= end_date]

    if merged.empty:
        print("No data available for the specified date range.")
        return

    # 5. Create plot
    print("Generating plot...")
    fig, ax1 = plt.subplots(figsize=(14, 7))
    
    # Plot stock price
    ax1.plot(merged['date'], merged['close'], 
             'b-', linewidth=2, label='Stock Price')
    
    # Create twin axis for sentiment
    ax2 = ax1.twinx()
    ax2.plot(merged['date'], merged['sentiment_ma'], 
             'r-', linewidth=2, label='7-day Sentiment Avg')
    
    # Format the plot
    format_plot(ax1, ax2, ticker, sentiment_start, sentiment_end)
    
    # Combine legends
    lines1, labels1 = ax1.get_legend_handles_labels()
    lines2, labels2 = ax2.get_legend_handles_labels()
    ax2.legend(lines1 + lines2, labels1 + labels2, loc='upper left')

    plt.tight_layout()
    plt.show()

# ─── ENTRY POINT ──────────────────────────────────────────────────────────────
if __name__ == "__main__":
    ticker = input("Enter ticker (e.g. AAPL): ").strip().upper()
    try:
        # Example: main(ticker, start_date='2025-04-15', end_date='2025-07-15')
        main(ticker)  # Use without date range for full history
    except Exception as e:
        print(f"Error: {e}")

In [ ]:
# visualize_sentiment_combined.py
import os
import sys
from pathlib import Path
from datetime import datetime, timedelta

import pandas as pd
import torch
import requests
import matplotlib.pyplot as plt
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# ─── resolve script_dir & add project root ────────────────────────────────────
try:
    script_dir = Path(__file__).parent
except NameError:
    script_dir = Path.cwd()

proj_root = script_dir.parent
if str(proj_root) not in sys.path:
    sys.path.insert(0, str(proj_root))

# ─── CONFIG ───────────────────────────────────────────────────────────────────
API_KEY = os.getenv("FMP_API_KEY")
if not API_KEY:
    raise RuntimeError("Missing FMP_API_KEY environment variable. "
                      "Please set it to your FinancialModelingPrep API key.")

MODEL_PATH = script_dir / "finbert-finetuned-final"
ART_DIR = script_dir / "articles"

# ─── HELPERS ──────────────────────────────────────────────────────────────────
def load_articles(ticker: str) -> pd.DataFrame:
    """Load news articles for given ticker"""
    path = ART_DIR / f"{ticker}.csv"
    if not path.exists():
        raise FileNotFoundError(f"No article CSV found at {path}")
    df = pd.read_csv(path, parse_dates=["publishedDate"])
    df["date"] = df["publishedDate"].dt.date
    return df

def analyze_sentiment(df: pd.DataFrame, tokenizer, model) -> pd.DataFrame:
    """Analyze sentiment of news articles"""
    recs = []
    for _, row in df.iterrows():
        text = row.get("content") or row.get("title", "")
        if not isinstance(text, str) or not text.strip():
            continue
        inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
        with torch.no_grad():
            logits = model(**inputs).logits
        probs = torch.softmax(logits, dim=1)[0]
        recs.append({
            "date": row["date"],
            "sentiment": probs[0].item() - probs[2].item(),  # positive - negative
            "title": row.get("title", ""),
            "source": row.get("site", "Unknown")
        })
    return pd.DataFrame(recs)

def fetch_prices(ticker: str) -> pd.DataFrame:
    """Fetch historical stock prices"""
    url = (
        f"https://financialmodelingprep.com/api/v3/"
        f"historical-price-full/{ticker}?apikey={API_KEY}"
    )
    resp = requests.get(url, timeout=10)
    if resp.status_code == 401:
        raise RuntimeError("Unauthorized fetching prices: check your FMP_API_KEY.")
    resp.raise_for_status()
    data = resp.json().get("historical", [])
    if not data:
        raise ValueError(f"No price history returned for {ticker}.")
    prices = pd.DataFrame(data)
    prices["date"] = pd.to_datetime(prices["date"])
    return prices[["date", "close"]]

# ─── MAIN ─────────────────────────────────────────────────────────────────────
def main(ticker: str, start_date=None, end_date=None):
    """Main analysis and plotting function"""
    # 1. Load model and tokenizer
    print("Loading sentiment model...")
    tokenizer = AutoTokenizer.from_pretrained(str(MODEL_PATH))
    model = AutoModelForSequenceClassification.from_pretrained(str(MODEL_PATH))
    model.eval()

    # 2. Score articles
    print("Loading and analyzing articles...")
    try:
        art_df = load_articles(ticker)
    except FileNotFoundError as e:
        print(e)
        return

    sent_df = analyze_sentiment(art_df, tokenizer, model)
    if sent_df.empty:
        print(f"No valid articles found for {ticker}.")
        return

    sentiment_start = sent_df['date'].min()
    sentiment_end = sent_df['date'].max()

    # 3. Calculate daily sentiment
    print("Calculating daily sentiment...")
    daily = (
        sent_df
        .groupby("date")["sentiment"]
        .mean()
        .reset_index()
        .assign(date=lambda df: pd.to_datetime(df["date"]))
    )

    # 4. Fetch and merge prices
    print("Fetching price data...")
    try:
        price_df = fetch_prices(ticker)
    except Exception as e:
        print(f"Error fetching prices: {e}")
        return

    merged = price_df.merge(daily, on="date", how="left")
    merged["sentiment"].fillna(0, inplace=True)  # Neutral for days without articles
    merged["sentiment_ma"] = merged["sentiment"].rolling(7, min_periods=1).mean()

    # Filter by date range if specified
    if start_date:
        start_date = pd.to_datetime(start_date)
        merged = merged[merged['date'] >= start_date]
    if end_date:
        end_date = pd.to_datetime(end_date)
        merged = merged[merged['date'] <= end_date]

    if merged.empty:
        print("No data available for the specified date range.")
        return

    # 5. Create plot in p2 style
    print("Generating plot...")
    fig, ax1 = plt.subplots(figsize=(14, 7))

    # Plot stock prices
    ax1.plot(merged['date'], merged['close'], 'b-', linewidth=2, label='Stock Price')
    ax1.set_xlabel('Date')
    ax1.set_ylabel('Stock Price', color='b')
    ax1.tick_params('y', colors='b')
    ax1.grid(True, linestyle='--', alpha=0.7)
    ax1.set_title(f'{ticker} Stock Price vs. News Sentiment')

    # Create second axis for sentiment
    ax2 = ax1.twinx()

    # Plot sentiment rolling average
    ax2.plot(merged['date'], merged['sentiment_ma'],
            'r-', linewidth=2,
            label='7-day Sentiment Avg')
    ax2.set_ylabel('Sentiment Score (7-day MA)', color='r')
    ax2.tick_params('y', colors='r')

    # Add horizontal line at 0 for neutral sentiment
    ax2.axhline(0, color='gray', linestyle='--', linewidth=0.8, label='Neutral')

    # Add markers for the actual sentiment period
    ax2.axvline(pd.to_datetime(sentiment_start), color='g', linestyle=':', alpha=0.7, label='Sentiment Start')
    ax2.axvline(pd.to_datetime(sentiment_end), color='r', linestyle=':', alpha=0.7, label='Sentiment End')

    # Combine legends from both axes
    lines1, labels1 = ax1.get_legend_handles_labels()
    lines2, labels2 = ax2.get_legend_handles_labels()
    ax2.legend(lines1 + lines2, labels1 + labels2, loc='upper left')

    plt.tight_layout()
    plt.show()

    # Show sample of the most positive/negative news (from p2)
    print("\nTop 3 Positive News:")
    print(sent_df.nlargest(3, 'sentiment')[['date', 'title', 'sentiment']])

    print("\nTop 3 Negative News:")
    print(sent_df.nsmallest(3, 'sentiment')[['date', 'title', 'sentiment']])

# ─── ENTRY POINT ──────────────────────────────────────────────────────────────
if __name__ == "__main__":
    ticker = input("Enter ticker (e.g. AAPL): ").strip().upper()
    try:
        # Example: main(ticker, start_date='2025-04-15', end_date='2025-07-15')
        main(ticker)  # Use without date range for full history
    except Exception as e:
        print(f"Error: {e}")

In [ ]:
import requests
import pandas as pd
import torch
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# 1. Load your custom model
model_path = "finbert-finetuned-final"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)
model.eval()

# 2. Fetch news from FMP with proper error handling
def fetch_news(symbol, api_key, days=3730):
    cutoff = datetime.utcnow() - timedelta(days=days)
    all_articles = []
    page = 0

    while True:
        url = f"https://financialmodelingprep.com/api/v3/stock_news?symbol={symbol}&page={page}&apikey={api_key}"
        try:
            response = requests.get(url)
            response.raise_for_status()
            news = response.json()

            if not news:
                break  # No more articles

            # Filter articles by date
            page_articles = []
            for article in news:
                try:
                    article_date = datetime.strptime(article['publishedDate'], "%Y-%m-%d %H:%M:%S")
                    if article_date < cutoff:
                        continue  # Skip articles older than cutoff
                    page_articles.append(article)
                except (KeyError, ValueError):
                    continue

            all_articles.extend(page_articles)

            # Check if we've reached the cutoff date
            if len(page_articles) < len(news):
                break  # This page contained articles beyond cutoff date

            page += 1

            # Safety limit to prevent infinite loops
            if page > 50:  # Max 50 pages (50,000 articles)
                print("Reached maximum page limit")
                break

        except requests.exceptions.RequestException as e:
            print(f"Error fetching page {page}: {e}")
            break

    print(f"Found {len(all_articles)} articles within {days} days")
    return all_articles

# 3. Analyze sentiment with YOUR model
def analyze_sentiment(articles):
    results = []
    for art in articles:
        try:
            # Use title if content is missing
            text = art.get('content', art.get('title', ''))
            if not text:
                continue

            inputs = tokenizer(text,
                              return_tensors="pt",
                              truncation=True,
                              max_length=512)
            with torch.no_grad():
                logits = model(**inputs).logits
            probs = torch.softmax(logits, dim=1)[0]

            results.append({
                'date': datetime.strptime(art['publishedDate'], "%Y-%m-%d %H:%M:%S").date(),
                'sentiment': probs[0].item() - probs[2].item(),  # positive - negative
                'title': art['title'],
                'source': art.get('site', 'Unknown')
            })
        except Exception as e:
            print(f"Error processing article '{art.get('title', '')}': {e}")

    return pd.DataFrame(results)

# 4. Fetch stock prices with error handling
def fetch_prices(symbol, api_key):
    url = f"https://financialmodelingprep.com/api/v3/historical-price-full/{symbol}?apikey={api_key}"
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()

        # Check if response contains historical data
        if 'historical' not in data:
            print(f"No price data found: {data}")
            return pd.DataFrame()

        prices = pd.DataFrame(data['historical'])
        prices['date'] = pd.to_datetime(prices['date'])
        return prices[['date', 'close']]

    except requests.exceptions.RequestException as e:
        print(f"Failed to fetch prices: {e}")
        return pd.DataFrame()

# 5. Main workflow
API_KEY = "API_KEY_HERE"  # Replace with your actual API key
SYMBOL = "AAPL"

# Fetch and process data
print("Fetching news...")
news = fetch_news(SYMBOL, API_KEY, days=3507)
print(f"Found {len(news)} articles")

print("Analyzing sentiment...")
sentiment_df = analyze_sentiment(news)
print(f"Processed {len(sentiment_df)} articles")

print("Fetching stock prices...")
prices_df = fetch_prices(SYMBOL, API_KEY)
print(f"Found {len(prices_df)} price records")

if sentiment_df.empty or prices_df.empty:
    print("Insufficient data for visualization")
else:
    # Aggregate daily sentiment
    daily_sentiment = sentiment_df.groupby('date')['sentiment'].mean().reset_index()
    daily_sentiment['date'] = pd.to_datetime(daily_sentiment['date'])

    # Merge with prices
    merged_df = pd.merge(
        prices_df,
        daily_sentiment,
        on='date',
        how='left'
    )

    # Filter to the period where we have sentiment data
    sentiment_start = daily_sentiment['date'].min()
    sentiment_end = daily_sentiment['date'].max()
    chart_start = sentiment_start - timedelta(days=30)
    chart_end = sentiment_end + timedelta(days=5)

    filtered_df = merged_df[
        (merged_df['date'] >= chart_start) &
        (merged_df['date'] <= chart_end)
    ]

    # Fill missing sentiment with 0 (neutral)
    filtered_df['sentiment'].fillna(0, inplace=True)

    # Add rolling average for sentiment
    filtered_df['sentiment_ma'] = filtered_df['sentiment'].rolling(7, min_periods=1).mean()

    # 6. Visualization - single plot with dual axes
    fig, ax1 = plt.subplots(figsize=(14, 7))

    # Plot stock prices
    ax1.plot(filtered_df['date'], filtered_df['close'], 'b-', linewidth=2, label='Stock Price')
    ax1.set_xlabel('Date')
    ax1.set_ylabel('Stock Price', color='b')
    ax1.tick_params('y', colors='b')
    ax1.grid(True, linestyle='--', alpha=0.7)
    ax1.set_title(f'{SYMBOL} Stock Price vs. News Sentiment')

    # Create second axis for sentiment
    ax2 = ax1.twinx()

    # Plot sentiment rolling average
    ax2.plot(filtered_df['date'], filtered_df['sentiment_ma'],
            'r-', linewidth=2,
            label='7-day Sentiment Avg')
    ax2.set_ylabel('Sentiment Score (7-day MA)', color='r')
    ax2.tick_params('y', colors='r')

    # Add horizontal line at 0 for neutral sentiment
    ax2.axhline(0, color='gray', linestyle='--', linewidth=0.8, label='Neutral')

    # Add markers for the actual sentiment period
    ax2.axvline(sentiment_start, color='g', linestyle=':', alpha=0.7, label='Sentiment Start')
    ax2.axvline(sentiment_end, color='r', linestyle=':', alpha=0.7, label='Sentiment End')

    # Combine legends from both axes
    lines1, labels1 = ax1.get_legend_handles_labels()
    lines2, labels2 = ax2.get_legend_handles_labels()
    ax2.legend(lines1 + lines2, labels1 + labels2, loc='upper left')

    plt.tight_layout()
    plt.show()

    # Show sample of the most positive/negative news
    print("\nTop 3 Positive News:")
    print(sentiment_df.nlargest(3, 'sentiment')[['date', 'title', 'sentiment']])

    print("\nTop 3 Negative News:")
    print(sentiment_df.nsmallest(3, 'sentiment')[['date', 'title', 'sentiment']])